In [0]:

import psycopg2
import datetime
# spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as F

def citus_row(start,end):
    def get_data_in_citus(start,end):
        citus_dsn_ = (
            "dbname='{db}' user='{user}' password='{password}' "
            "host='{host}' port='{port}'".format(
                db="aa_citus_db",
                user="citus_bdp_usage_qa",
                host="10.2.10.132",
                password="dNzWtSV3pKTx",
                port=5432
            )
        )
        count_sql = "SELECT date,count(*),count(distinct(app_price_type_id, purchase_type_id,category_id,device_code,country_code,date,granularity)),cast(SUM(est_market_size_download) as bigint) AS sum_est_market_size_download,cast(SUM(est_market_size_revenue) as bigint)AS sum_est_market_size_revenue FROM store.store_market_size_fact_v1 WHERE date >='{}' AND date<='{}' GROUP BY date".format(start,end)
        db_data = query(citus_dsn_, count_sql)
        
        return db_data
    def query(dsn, sql):
        with psycopg2.connect(dsn) as conn:
            conn.autocommit = True
            with conn.cursor() as cur:
                cur.execute(sql)
                result = cur.fetchall()
                conn.commit()
        return result
    result = get_data_in_citus(start,end)
    return [Row(date=r[0], total_row=r[1], distinct_metrics=r[2], sum_est_market_size_download=r[3], sum_est_market_size_revenue=r[4]) for r in result]
def generate_citus_result(spark, citus_data):
    schema = StructType([
    StructField("date", StringType(), True),
    StructField("total_row", LongType(), True),
    StructField("distinct_metrics", LongType(), True),
    StructField("sum_est_market_size_download", LongType(), True),
    StructField("sum_est_market_size_revenue", LongType(), True)
    ])
    df_3 = spark.createDataFrame(citus_data, schema)
    df_3.createOrReplaceTempView("citus_data")
    return spark.sql("select sum(total_row),sum(sum_est_market_size_download),sum(sum_est_market_size_revenue) from citus_data")

print("================online=======================")
online_df = generate_citus_result(spark,citus_row("2020-08-01","2020-08-31"))
online_df.show()

print("================ADL=========================")

sample_data_df=spark.read.format("parquet").load("s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/snowflake_v1/store.market-download-revenue.v1/fact/")
sample_data_df.createOrReplaceTempView("sample_data")

spark.sql("SELECT SUM(est_market_size_download) as sum_est_market_size_download, SUM(est_market_size_revenue) as sum_est_market_size_revenue FROM sample_data").show()
print(sample_data_df.count())

In [0]:

%sh
PGPASSWORD='dNzWtSV3pKTx' psql -h 10.2.10.132 -U citus_bdp_usage_qa -d aa_citus_db -p 5432 << EOF 
SELECT date,count(*),count(distinct(app_price_type_id, purchase_type_id,category_id,device_code,country_code,date,granularity)) FROM store.store_market_size_fact_v1  WHERE date between '2019-12-01' AND '2019-12-02' GROUP BY date;